In [1]:
# ! wget http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip

In [2]:
# ! ls

In [3]:
# ! apt install unzip
# ! unzip cornell_movie_dialogs_corpus.zip

In [4]:
# ! ls

In [5]:
# ! wget http://opus.nlpl.eu/download.php?f=OpenSubtitles/v2018/mono/OpenSubtitles.raw.en.gz

In [6]:
# ! ls

In [7]:
# !tar -xvf  'download.php?f=OpenSubtitles%2Fv2018%2Fmono%2FOpenSubtitles.raw.en.gz' -C 'data'

In [8]:
! wget https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/mono/en.txt.gz

--2020-06-19 18:27:19--  https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/mono/en.txt.gz
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3663376519 (3.4G) [application/gzip]
Saving to: ‘en.txt.gz’

en.txt.gz           100%[===================>]   3.41G  22.0MB/s    in 2m 39s  

2020-06-19 18:29:58 (22.0 MB/s) - ‘en.txt.gz’ saved [3663376519/3663376519]



In [9]:
! apt install gunzip
! gunzip en.txt.gz

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package gunzip


In [11]:
# % cd ..

In [12]:
# ! wget https://github.com/PolyAI-LDN/conversational-datasets/blob/master/opensubtitles/create_data.py

In [43]:
import os
import re
from os import path
import tensorflow as tf
import numpy as np

In [44]:
def _should_skip(line, min_length, max_length):
    """Whether a line should be skipped depending on the length."""
    return len(line) < min_length or len(line) > max_length

In [45]:
def create_example(previous_lines, line, file_id):
    """Creates examples with multi-line context
    The examples will include:
        file_id: the name of the file where these lines were obtained.
        response: the current line text
        context: the previous line text
        context/0: 2 lines before
        context/1: 3 lines before, etc.
    """

    example = {
        'file_id': file_id,
        'context': previous_lines[-1],
        'response': line,
    }
    example['file_id'] = file_id
    #print(file_id)
    example['context'] = previous_lines[-1]
    #print(previous_lines[-1])
    extra_contexts = previous_lines[:-1]
    example.update({
        'context/{}'.format(i): context
        for i, context in enumerate(extra_contexts[::-1])
    })

    return example

In [46]:
def _preprocess_line(line):
    #line = line.decode("utf-8")

    # Remove the first word if it is followed by colon (speaker names)
    # NOTE: this wont work if the speaker's name has more than one word
    line = re.sub('(?:^|(?:[.!?]\\s))(\\w+):', "", line)

    # Remove anything between brackets (corresponds to acoustic events).
    line = re.sub("[\\[(](.*?)[\\])]", "", line)

    # Strip blanks hyphens and line breaks
    line = line.strip(" -\n")

    return line

In [48]:
def _create_examples_from_file(file_name,file_id, min_length=0, max_length=24,
                               num_extra_contexts=2):
   # _, file_id = path.split(file_name)
    #print(file_id,"#")
    previous_lines = []
    for line in open(file_name):
        line = _preprocess_line(line)
        
        if not line:
            continue

        should_skip = _should_skip(
            line,
            min_length=min_length,
            max_length=max_length)

        if previous_lines:
            should_skip |= _should_skip(
                previous_lines[-1],
                min_length=min_length,
                max_length=max_length)

            if not should_skip:
                yield create_example(previous_lines, line, file_id)

        previous_lines.append(line)
        if len(previous_lines) > num_extra_contexts + 1:
            del previous_lines[0]

In [49]:
def _features_to_serialized_tf_example(features):
    """Convert a string dict to a serialized TF example.
    The dictionary maps feature names (strings) to feature values (strings).
    """
    #print("hello")
    example = tf.train.Example()
    for feature_name, feature_value in features.items():
        example.features.feature[feature_name].bytes_list.value.append(
            feature_value.encode("utf-8"))
    return example.SerializeToString()


def _shuffle_examples(examples):
    examples |= ("add random key" >> beam.Map(
        lambda example: (uuid.uuid4(), example)))
    examples |= ("group by key" >> beam.GroupByKey())
    examples |= ("get shuffled values" >> beam.FlatMap(lambda t: t[1]))
    return examples

In [50]:
# % cd ..

In [51]:
# ! ls

In [52]:
# ! pwd

In [53]:
# ! ( head -1000000 en.txt ; ) > million.txt

In [62]:
test=_create_examples_from_file(file_name='en.txt',file_id='en.txt', num_extra_contexts=0)

In [88]:
alpha=[]
sent=[]
count=0
num_examples = 10**1
for x in test:
  count+=1
  if count<num_examples:
    alpha.append(x)  
    sent.append  
  else:
    break

In [89]:
len(alpha)

9

In [91]:
X_train[0]

"We're not?"

In [92]:
dat=open('en.txt')
sents=[]
for line in dat:
  sents.append(_preprocess_line(line))
  if len(sents)>num_examples-1:
    break

In [93]:
len(sents)

10

In [99]:
import itertools
import collections
import numpy as np
import nltk
nltk.download('punkt')
SENT_START_TOKEN = "SENTENCE_START"
SENT_END_TOKEN = "SENTENCE_END"
UNKNOWN_TOKEN = "UNKNOWN_TOKEN"
PADDING_TOKEN = "PADDING"


def tokenize_text(text_lines):
    """
    Split text into sentences, append start and end tokens to each and tokenize
    :param text_lines: list of text lines or list of length one containing all text
    :return: list of sentences
    """
    sentences = itertools.chain(*[nltk.sent_tokenize(line.lower()) for line in text_lines])
    sentences = ["{} {} {}".format(SENT_START_TOKEN, x, SENT_END_TOKEN) for x in sentences]
    tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]
    return tokenized_sentences

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [100]:
t_sents=tokenize_text(sents)

In [101]:
def get_words_mappings(tokenized_sentences, vocabulary_size):
    # Using NLTK
    # frequence = nltk.FreqDist(itertools.chain(*tokenized_sentences))
    # vocab = frequence.most_common(vocabulary_size)

    # Using basic counter
    counter = collections.Counter(itertools.chain(*tokenized_sentences))
    vocab = counter.most_common(vocabulary_size)
    index_to_word = [x[0] for x in vocab]
    # Add padding for index 0
    index_to_word.insert(0, PADDING_TOKEN)
    # Append unknown token (with index = vocabulary size + 1)
    index_to_word.append(UNKNOWN_TOKEN)
    word_to_index = dict([(w, i) for i, w in enumerate(index_to_word)])
    return index_to_word, word_to_index

In [117]:
vocabulary_size=10**4

In [102]:
id2w,w2id=get_words_mappings(t_sents,vocabulary_size=vocabulary_size)

In [136]:
tok_sents=[]
for sent in t_sents:
  sent=[w2id[w] for w in sent]
  tok_sents.append(sent)

In [142]:
print(len(tok_sents))

10


In [143]:
X_train = np.array([tok_sents[i] for i in range(0, len(tok_sents)-1)])
Y_train = np.array([tok_sents[i] for i in range(1,len(tok_sents))])

In [145]:
print(len(X_train),len(Y_train))
print(X_train[0],Y_train[0])

9 9
[1, 15, 6, 16, 17, 2] [1, 18, 6, 19, 20, 2]


In [114]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2020-06-19 19:23:44--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-06-19 19:23:44--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-06-19 19:23:44--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [146]:
glove_dir = './'

embeddings_index = {} #initialize dictionary
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [147]:
embedding_dim = 100
num_words= vocabulary_size
embedding_matrix = np.zeros((num_words, embedding_dim)) #create an array of zeros with word_num rows and embedding_dim columns
for i, word in enumerate(id2w):
    embedding_vector = embeddings_index.get(word)
    if i < num_words:
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

In [148]:
from keras import initializers, models, regularizers
from keras.layers import Dense, Dropout, Embedding, SeparableConv1D, MaxPooling1D, GlobalAveragePooling1D

In [121]:
toyModel = models.Sequential()
toyModel.add(Embedding(num_words, 
                    embedding_dim, 
                    input_length=24,
                      weights=[embedding_matrix],
                      trainable=False))

In [123]:
toyModel.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 24, 100)           1000000   
Total params: 1,000,000
Trainable params: 0
Non-trainable params: 1,000,000
_________________________________________________________________
